In [1]:
import pandas as pd
from tensorflow import keras
from os import path
import pickle


In [2]:
from proteinbert.finetuning import encode_train_and_valid_sets, encode_dataset
from proteinbert import OutputType, OutputSpec, evaluate_by_len, load_pretrained_model

In [3]:
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, \
finetune, evaluate_by_len

from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [4]:
import wandb
from wandb.keras import WandbCallback

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [6]:
DATA_DIR = "../../data/"
VALID_SPLIT_SEED = 333

In [7]:
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

In [8]:
pretrained_model_generator, input_encoder = load_pretrained_model("../../data/protein_bert/", "epoch_92400_sample_23500000.pkl")

In [9]:
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

In [10]:
wandb.init(project=f"Old_split_finetune", entity="kvetab")

wandb: Currently logged in as: kvetab (use `wandb login --relogin` to force relogin)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


# Split 4a

In [11]:
train_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/crossval/chen_4_a.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/crossval/chen_4_b.csv"), index_col=0)
train_data.head()

,Antibody_ID,heavy,light,Y,cluster
0,6aod,EVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLE...,DIVMTKSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKR...,0,165
2,6and,EVQLVESGGGLVQPGGSLRLSCAASGYEFSRSWMNWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRSSQSIVHSVGNTFLEWYQQKPG...,1,135
9,3esu,EVQLQQSGPELVKPGASVKISCKDSGYAFSSSWMNWVKQRPGQGPE...,DIVLIQSTSSLSASLGDRVTISCRASQDIRNYLNWYQQKPDGTVKL...,0,46
10,3esv,EVQLQQSGPELVKPGASVKISCKDSGYAFNSSWMNWVKQRPGQGLE...,DIQMTQTTSSLSASLGDRVTVSCRASQDIRNYLNWYQQKPDGTVKF...,0,46
11,3et9,EVQLQQSGPELVKPGASVKISCKDSGYAFSSSWMNWVKQRPGQGPE...,DIVLIQSTSSLSASLGDRVTISCRASQDIRNYLNWYQQKPDGTVKL...,0,46


In [12]:
train_data["seq"] = train_data["heavy"] + train_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]

In [13]:
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 3, factor = 0.25, min_lr = 1e-07, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 6, restore_best_weights = True),
    WandbCallback()
]

In [14]:
epoch_num = 50
batch_size = 128
learning_rate = 1e-4

In [15]:
wandb.config = {
      "learning_rate": learning_rate,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }

In [16]:
##### Validating using test data!!! #####

In [17]:
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], test_data['seq'], test_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)

[2022_04_20-11:55:49] Training set: Filtered out 0 of 585 (0.0%) records of lengths exceeding 510.
[2022_04_20-11:55:49] Validation set: Filtered out 0 of 733 (0.0%) records of lengths exceeding 510.
[2022_04_20-11:55:49] Training with frozen pretrained layers...


2022-04-20 11:55:49.746183: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-20 11:55:50.305130: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9656 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:41:00.0, compute capability: 7.5
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-04-20 11:55:52.250985: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are en

Epoch 1/50


2022-04-20 11:55:59.787175: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 7605


5/5 [==============================] - 11s 791ms/step - loss: 0.8133 - val_loss: 0.6563


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/50
5/5 [==============================] - 1s 239ms/step - loss: 0.6111 - val_loss: 0.7012
Epoch 3/50
5/5 [==============================] - 1s 239ms/step - loss: 0.5588 - val_loss: 0.5636
Epoch 4/50
5/5 [==============================] - 1s 236ms/step - loss: 0.5089 - val_loss: 0.5047
Epoch 5/50
5/5 [==============================] - 1s 239ms/step - loss: 0.4533 - val_loss: 0.4726
Epoch 6/50
5/5 [==============================] - 1s 237ms/step - loss: 0.4418 - val_loss: 0.4622
Epoch 7/50
5/5 [==============================] - 1s 238ms/step - loss: 0.4370 - val_loss: 0.4592
Epoch 8/50
5/5 [==============================] - 1s 239ms/step - loss: 0.4249 - val_loss: 0.4527
Epoch 9/50
5/5 [==============================] - 1s 238ms/step - loss: 0.4141 - val_loss: 0.4612
Epoch 10/50
5/5 [==============================] - 1s 238ms/step - loss: 0.4101 - val_loss: 0.4456
Epoch 11/50
5/5 [==============================] - 1s 240ms/step - loss: 0.3975 - val_loss: 0.4485
Epoch 12/50
5/5 [=

In [18]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)

In [19]:
print('Test-set performance:')
display(results)

print('Confusion matrix:')
display(confusion_matrix)

Test-set performance:


,# records,AUC
Model seq len,,
512,733,0.785587
All,733,0.785587


Confusion matrix:


,0,1
0,496,86
1,78,73


In [20]:
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

0.47096774193548385

In [22]:
mod = model_generator.create_model(seq_len = 512)
mod.save(path.join(DATA_DIR, "protein_bert/2022_04_09_01_x"))

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-04-09 11:32:18.361248: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_04_09_01/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/2022_04_09_01/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [10]:
patience = [(3,3), (4,3), (6,3), (8,4)]
learning_rate = [1e-5, 5e-5, 1e-4, 5e-4]
prepro = ["scaling"]

In [23]:
def finetune_by_settings(patience, lr, project_name):
    wandb.init(project=project_name, entity="kvetab")
    model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)
    training_callbacks = [
        keras.callbacks.ReduceLROnPlateau(patience = patience[1], factor = 0.25, min_lr = 1e-07, verbose = 1),
        keras.callbacks.EarlyStopping(patience = patience[0], restore_best_weights = True),
        WandbCallback()
    ]
    epoch_num = 100
    batch_size = 128
    wandb.config = {
          "learning_rate": lr,
          "epochs": epoch_num * 2,
          "batch_size": batch_size
        }
    print(type(lr))
    print(lr)
    finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], test_data['seq'], test_data['Y'], \
            seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = lr, begin_with_frozen_pretrained_layers = True, \
            lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 512, final_lr = lr / 10, callbacks = training_callbacks)
    
    results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
            start_seq_len = 512, start_batch_size = 32)
    print('Confusion matrix:')
    display(confusion_matrix)
    fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
    f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
    print(f1)
    
    mod = model_generator.create_model(seq_len = 512)
    mod.save(path.join(DATA_DIR, f"protein_bert/4b/2022_04_20_{patience[0]}_{patience[1]}_{lr}"))

In [24]:
for pat in patience:
    for lr in learning_rate:
        finetune_by_settings(pat, lr, "Split 4a")

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███▁▁▂▂▂▂▃▃▃▃▁
loss,█▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
lr,███████████████████▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▇█▃▂▂▂▂▂▂▂▂▁▂▁▁▁▃▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
best_epoch,11
best_val_loss,0.41962
epoch,0
loss,0.29309
lr,1e-05
val_loss,0.41963


wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


<class 'float'>
1e-05
[2022_04_20-12:01:14] Training set: Filtered out 0 of 585 (0.0%) records of lengths exceeding 510.
[2022_04_20-12:01:14] Validation set: Filtered out 0 of 733 (0.0%) records of lengths exceeding 510.
[2022_04_20-12:01:14] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
5/5 [==============================] - 10s 710ms/step - loss: 0.9246 - val_loss: 0.5503


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/100
5/5 [==============================] - 1s 242ms/step - loss: 0.5783 - val_loss: 0.7069
Epoch 3/100
2/5 [===========>..................] - ETA: 0s - loss: 0.6474

KeyboardInterrupt: 

In [36]:
train_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/crossval/chen_4_b.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/crossval/chen_4_a.csv"), index_col=0)
train_data["seq"] = train_data["heavy"] + train_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]

for pat in patience:
    for lr in learning_rate:
        finetune_by_settings(pat, lr, "Split 4b")

wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


<class 'float'>
1e-05
[2022_04_09-17:22:57] Training set: Filtered out 0 of 654 (0.0%) records of lengths exceeding 510.
[2022_04_09-17:22:57] Validation set: Filtered out 0 of 637 (0.0%) records of lengths exceeding 510.
[2022_04_09-17:22:57] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
6/6 [==============================] - 54s 581ms/step - loss: 0.8790 - val_loss: 0.4708


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


...

,0,1
0,458,71
1,66,42


0.38009049773755654


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/4b/2022_04_09_8_4_0.0005/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/4b/2022_04_09_8_4_0.0005/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# 5x2cv

In [24]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
import os
import csv

In [26]:
def test_trained_model(model, dir_name, model_name, x_test, y_test, x_tap, y_tap, lr, patience_lr, patience_stop):
    y_pred = model.predict(x_test, batch_size=32)
    y_pred_classes = (y_pred >= 0.5)
    metric_dict = {
        "f1": float(metrics.f1_score(y_test, y_pred_classes)),
        "acc": float(metrics.accuracy_score(y_test, y_pred_classes)),
        "mcc": float(metrics.matthews_corrcoef(y_test, y_pred_classes)),
        "auc": float(metrics.roc_auc_score(y_test, y_pred_classes)),
        "precision": float(metrics.precision_score(y_test, y_pred_classes)),
        "recall": float(metrics.recall_score(y_test, y_pred_classes))
    }
    filename = path.join(DATA_DIR, f"evaluations/protein_bert/{dir_name}/{model_name}_preds.csv")
    str_preds = [str(int(pred)) for pred in y_pred_classes]
    with open(filename, "wt") as f:
        f.write(",".join(str_preds) + "\n")
        
    filename_sum = path.join(DATA_DIR, f"evaluations/protein_bert/{dir_name}/all.csv")
    line = ["protein_bert", patience_stop, patience_lr, lr, metric_dict["f1"], metric_dict["mcc"], metric_dict["acc"],metric_dict["precision"],metric_dict["recall"],metric_dict["auc"]]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)
    
    
    y_pred = model.predict(x_tap, batch_size=32)
    y_pred_classes = (y_pred >= 0.5)
    metric_dict = {
        "f1": float(metrics.f1_score(y_tap, y_pred_classes)),
        "acc": float(metrics.accuracy_score(y_tap, y_pred_classes)),
        "mcc": float(metrics.matthews_corrcoef(y_tap, y_pred_classes)),
        "auc": float(metrics.roc_auc_score(y_tap, y_pred_classes)),
        "precision": float(metrics.precision_score(y_tap, y_pred_classes)),
        "recall": float(metrics.recall_score(y_tap, y_pred_classes))
    }
        
    filename_sum = path.join(DATA_DIR, f"evaluations/protein_bert/{dir_name}/tap.csv")
    line = ["protein_bert", patience_stop, patience_lr, lr, metric_dict["f1"], metric_dict["mcc"], metric_dict["acc"],metric_dict["precision"],metric_dict["recall"],metric_dict["auc"]]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)

In [20]:
def finetune_by_settings_and_data(patience, lr, project_name, train_data, test_data, valid_data, tap_X, tap_Y, dir_name):
    wandb.init(project=project_name, entity="kvetab")
    model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)
    training_callbacks = [
        keras.callbacks.ReduceLROnPlateau(patience = patience[1], factor = 0.25, min_lr = 1e-07, verbose = 1),
        keras.callbacks.EarlyStopping(patience = patience[0], restore_best_weights = True),
        WandbCallback()
    ]
    epoch_num = 100
    batch_size = 128
    wandb.config = {
          "learning_rate": lr,
          "epochs": epoch_num * 2,
          "batch_size": batch_size
        }
    print(type(lr))
    print(lr)
    finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
            seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = lr, begin_with_frozen_pretrained_layers = True, \
            lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 512, final_lr = lr / 10, callbacks = training_callbacks)
    
    
    mod = model_generator.create_model(seq_len = 512)
    mod_name = f"2022_04_21_{patience[0]}_{patience[1]}_{lr}"
    mod.save(path.join(DATA_DIR, f"protein_bert/{dir_name}/{mod_name}"))
    
    encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')
    test_X, test_Y, test_sample_weigths = encoded_test_set

    test_trained_model(mod, dir_name, mod_name, test_X, test_Y, tap_X, tap_Y, lr, patience[1], patience[0])

In [27]:
seq_len = 512
tap_data = pd.read_csv(path.join(DATA_DIR, "tap/tap_not_in_chen.csv"))
tap_data["seq"] = tap_data["heavy"] + tap_data["light"]
encoded_tap_set = encode_dataset(tap_data["seq"], tap_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
        dataset_name = 'Tap set')
tap_X, tap_Y, test_sample_weigths = encoded_tap_set
for seed in [4, 18, 27, 36, 42]:
    train = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_a.csv"), index_col=0)
    test = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_b.csv"), index_col=0)
    train["seq"] = train["heavy"] + train["light"]
    test["seq"] = test["heavy"] + test["light"]
    valid_1, test_1 = train_test_split(
        test, test_size=0.6, random_state=VALID_SPLIT_SEED, stratify=test["Y"]
    )
    valid_2, test_2 = train_test_split(
        train, test_size=0.6, random_state=VALID_SPLIT_SEED, stratify=train["Y"]
    )
    
    for pat in patience:
        for lr in learning_rate:
            finetune_by_settings_and_data(pat, lr, f"Split {seed}a", train, test_1, valid_1, tap_X, tap_Y, f"{seed}a")
            finetune_by_settings_and_data(pat, lr, f"Split {seed}b", test, test_2, valid_2, tap_X, tap_Y, f"{seed}b")

[2022_04_21-13:40:17] Tap set: Filtered out 0 of 184 (0.0%) records of lengths exceeding 510.


epoch,▁▂▂▃▃▄▅▅▆▆▇▇█▁▂▂▃▁
loss,█▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
lr,█████████████▁▁▁▁▁
val_loss,▅█▂▅▂▂▁▂▁▁▂▁▁▁▁▁▁▁
best_epoch,9
best_val_loss,0.44473
epoch,0
loss,0.36251
lr,0.0
val_loss,0.44898


wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


<class 'float'>
1e-05
[2022_04_21-13:40:35] Training set: Filtered out 0 of 637 (0.0%) records of lengths exceeding 510.
[2022_04_21-13:40:35] Validation set: Filtered out 0 of 261 (0.0%) records of lengths exceeding 510.
[2022_04_21-13:40:35] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


...

6/6 [==============================] - 10s 628ms/step - loss: 0.3134 - val_loss: 0.4747


INFO:tensorflow:Assets written to: ../../data/protein_bert/42b/2022_04_21_8_4_0.0005/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/42b/2022_04_21_8_4_0.0005/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


[2022_04_21-19:43:48] Test set: Filtered out 0 of 380 (0.0%) records of lengths exceeding 510.


wandb: Network error (ReadTimeout), entering retry loop.


# Load Model

In [28]:
#from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn import metrics
import os
import csv

In [29]:
seq_len = 512
encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')

[2022_04_20-17:24:46] Test set: Filtered out 0 of 733 (0.0%) records of lengths exceeding 510.


In [22]:
test_X, test_Y, test_sample_weigths = encoded_test_set

In [35]:
tap_data = pd.read_csv(path.join(DATA_DIR, "tap/TAP_data.csv"))
tap_data.head()

,Antibody_ID,heavy,light,CDR_length,PSH,PPC,PNC,SFvCSP,Y
0,Abagovomab,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLD...,DIELTQSPASLSASVGETVTITCQASENIYSYLAWHQQKQGKSPQL...,46,129.7603,0.0000,0.0000,16.32,1
1,Abituzumab,QVQLQQSGGELAKPGASVKVSCKASGYTFSSFWMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDISNYLAWYQQKPGKAPKL...,45,115.9106,0.0954,0.0421,-3.10,1
2,Abrilumab,QVQLVQSGAEVKKPGASVKVSCKVSGYTLSDLSIHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQQKPGKAPKL...,45,109.6995,0.0000,0.8965,-4.00,1
3,Actoxumab,QVQLVESGGGVVQPGRSLRLSCAASGFSFSNYGMHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQHKPGKAPKL...,49,112.6290,0.0000,1.1247,3.10,1
4,Adalimumab,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQGIRNYLAWYQQKPGKAPKL...,48,111.2512,0.0485,1.1364,-19.50,1


In [36]:
tap_data["seq"] = tap_data["heavy"] +  tap_data["light"]

In [37]:
encoded_tap_set = encode_dataset(tap_data["seq"], tap_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'TAP set')

[2022_04_20-17:25:41] TAP set: Filtered out 0 of 241 (0.0%) records of lengths exceeding 510.


In [38]:
tap_X, tap_Y, tap_sample_weigths = encoded_tap_set

In [23]:
model_path = path.join(DATA_DIR, "protein_bert/2022_03_30__05.pkl")
model = keras.models.load_model(model_path)

In [24]:
y_pred = model.predict(test_X, batch_size=32)
y_pred_classes = (y_pred >= 0.5)
f1_score(test_Y, y_pred_classes)

0.6486486486486486

In [29]:
y_pred = model.predict(tap_X, batch_size=32)
y_pred_classes = (y_pred >= 0.5)
f1_score(tap_Y, y_pred_classes)

0.34285714285714286

In [30]:
accuracy_score(tap_Y, y_pred_classes)

0.23651452282157676

In [30]:
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
seq_len = 512
encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')
test_X, test_Y, test_sample_weigths = encoded_test_set

[2022_04_20-17:24:54] Test set: Filtered out 0 of 733 (0.0%) records of lengths exceeding 510.


In [31]:
def test_model(model_name, dir_name, x_test, y_test, x_tap, y_tap):
    model_path = path.join(DATA_DIR, f"protein_bert/{dir_name}/{model_name}")
    model = keras.models.load_model(model_path)
    parts = model_name[11:].split("_")
    patience_stop = parts[0]
    patience_lr = parts[1]
    lr = parts[2]
    
    y_pred = model.predict(x_test, batch_size=32)
    y_pred_classes = (y_pred >= 0.5)
    #print(f"Model {model_name}")
    #print(f"Test F1: {f1}")
    metric_dict = {
        "f1": float(metrics.f1_score(y_test, y_pred_classes)),
        "acc": float(metrics.accuracy_score(y_test, y_pred_classes)),
        "mcc": float(metrics.matthews_corrcoef(y_test, y_pred_classes)),
        "auc": float(metrics.roc_auc_score(y_test, y_pred_classes)),
        "precision": float(metrics.precision_score(y_test, y_pred_classes)),
        "recall": float(metrics.recall_score(y_test, y_pred_classes))
    }
    filename = path.join(DATA_DIR, f"evaluations/protein_bert/{dir_name}/{model_name}_preds.csv")
    str_preds = [str(int(pred)) for pred in y_pred_classes]
    with open(filename, "wt") as f:
        f.write(",".join(str_preds) + "\n")
        
    filename_sum = path.join(DATA_DIR, f"evaluations/protein_bert/{dir_name}/all.csv")
    line = ["protein_bert", patience_stop, patience_lr, lr, metric_dict["f1"], metric_dict["mcc"], metric_dict["acc"],metric_dict["precision"],metric_dict["recall"],metric_dict["auc"]]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)
    
    
    y_pred = model.predict(x_tap, batch_size=32)
    y_pred_classes = (y_pred >= 0.5)
    acc = accuracy_score(y_tap, y_pred_classes)
    metric_dict = {
        "f1": float(metrics.f1_score(y_tap, y_pred_classes)),
        "acc": float(metrics.accuracy_score(y_tap, y_pred_classes)),
        "mcc": float(metrics.matthews_corrcoef(y_tap, y_pred_classes)),
        "auc": float(metrics.roc_auc_score(y_tap, y_pred_classes)),
        "precision": float(metrics.precision_score(y_tap, y_pred_classes)),
        "recall": float(metrics.recall_score(y_tap, y_pred_classes))
    }
        
    filename_sum = path.join(DATA_DIR, f"evaluations/protein_bert/{dir_name}/tap.csv")
    line = ["protein_bert", patience_stop, patience_lr, lr, metric_dict["f1"], metric_dict["mcc"], metric_dict["acc"],metric_dict["precision"],metric_dict["recall"],metric_dict["auc"]]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)

In [57]:
seed = 4
split = "a"
train_data = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_a.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_b.csv"), index_col=0)
train_data["seq"] = train_data["heavy"] + train_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]

encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')
test_X, test_Y, test_sample_weigths = encoded_test_set

test_model("2022_04_09_3_3_0.0001", "4a", test_X, test_Y, tap_X, tap_Y)

[2022_04_10-09:53:46] Test set: Filtered out 0 of 654 (0.0%) records of lengths exceeding 510.


In [41]:
seeds = [4] # [ 18, 27, 36, 42] # 4

In [33]:
seq_len = 512

In [42]:
for seed in seeds:
    train_data = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_a.csv"), index_col=0)
    test_data = pd.read_csv(path.join(DATA_DIR, f"chen/deduplicated/crossval/chen_{seed}_b.csv"), index_col=0)
    train_data["seq"] = train_data["heavy"] + train_data["light"]
    test_data["seq"] = test_data["heavy"] + test_data["light"]
    encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')
    test_X, test_Y, test_sample_weigths = encoded_test_set
    encoded_train_set = encode_dataset(train_data["seq"], train_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Train set')
    train_X, train_Y, test_sample_weigths = encoded_train_set
    
    for split in ["a", "b"]:
        for model in os.listdir(path.join(DATA_DIR, f"protein_bert/{seed}{split}")):
            if model.startswith("2022_04_20"):
                if split == "a":
                    test_model(model, f"{seed}{split}", test_X, test_Y, tap_X, tap_Y)
                else:
                    test_model(model, f"{seed}{split}", train_X, train_Y, tap_X, tap_Y)
                    
                

[2022_04_20-19:39:32] Test set: Filtered out 0 of 733 (0.0%) records of lengths exceeding 510.
[2022_04_20-19:39:32] Train set: Filtered out 0 of 585 (0.0%) records of lengths exceeding 510.


# Full train data

In [31]:
from sklearn.model_selection import train_test_split

In [88]:
train_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train.csv"), index_col=0)
test_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test.csv"), index_col=0)
train_data["seq"] = train_data["heavy"] + train_data["light"]
test_data["seq"] = test_data["heavy"] + test_data["light"]
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=39)

In [92]:
for pat in patience:
    for lr in learning_rate:
        finetune_by_settings_and_data(pat, lr, f"Full", train_data, test_data, f"old_full")

epoch,▁▁▂▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇██▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇
loss,█▅▄▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
lr,███████▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▂▂▂▂▂▁▂▂▁▁▂▁▂▁▁▁▁▁▁▁▂▁▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁
best_epoch,13
best_val_loss,0.45486
epoch,0
loss,0.32702
lr,1e-05
val_loss,0.46158


wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


<class 'float'>
1e-05
[2022_04_15-14:37:59] Training set: Filtered out 0 of 1338 (0.0%) records of lengths exceeding 510.
[2022_04_15-14:37:59] Validation set: Filtered out 0 of 120 (0.0%) records of lengths exceeding 510.
[2022_04_15-14:37:59] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
11/11 [==============================] - 10s 286ms/step - loss: 0.8872 - val_loss: 0.7528


...

,0,1
0,93,1
1,18,8


0.45714285714285713


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


INFO:tensorflow:Assets written to: ../../data/protein_bert/old_full/2022_04_09_8_4_0.0005/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/old_full/2022_04_09_8_4_0.0005/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [51]:
encoded_test_set = encode_dataset(test_data["seq"], test_data["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set')

[2022_04_15-12:45:47] Test set: Filtered out 0 of 260 (0.0%) records of lengths exceeding 510.


In [52]:
test_X, test_Y, test_sample_weigths = encoded_test_set

In [93]:
encoded_test_set_old = encode_dataset(test_data_old["seq"], test_data_old["Y"], input_encoder, OUTPUT_SPEC, seq_len = seq_len, needs_filtering = True, \
            dataset_name = 'Test set old')
test_X_old, test_Y_old, test_sample_weigths_old = encoded_test_set_old

[2022_04_15-15:27:54] Test set old: Filtered out 0 of 119 (0.0%) records of lengths exceeding 510.


In [54]:
for model in os.listdir(path.join(DATA_DIR, f"protein_bert/full")):
    if model.startswith("2022_04_09"):
        test_model(model, f"full", test_X, test_Y, tap_X, tap_Y)

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [95]:
for model in os.listdir(path.join(DATA_DIR, f"protein_bert/old_full")):
    if model.startswith("2022_04_09"):
        test_model(model, f"old_full", test_X_old, test_Y_old, tap_X, tap_Y)

In [59]:
model_name = "2022_04_09_6_3_0.0001"
model_path = path.join(DATA_DIR, f"protein_bert/full/{model_name}")
model = keras.models.load_model(model_path)
parts = model_name[11:].split("_")
patience_stop = parts[0]
patience_lr = parts[1]
lr = parts[2]

y_pred = model.predict(test_X, batch_size=32)
y_pred_classes = (y_pred >= 0.5)

In [61]:
metrics.f1_score(test_Y, y_pred_classes)

0.15384615384615383

In [62]:
metrics.confusion_matrix(test_Y, y_pred_classes)

array([[200,   6],
       [ 49,   5]])

In [80]:
len(train_data)

1291

In [89]:
wandb.init(project="Easter", entity="kvetab")
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
    get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)
training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 3, factor = 0.25, min_lr = 1e-07, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 6, restore_best_weights = True),
    WandbCallback()
]
epoch_num = 100
batch_size = 128
lr = 1e-4
wandb.config = {
      "learning_rate": lr,
      "epochs": epoch_num * 2,
      "batch_size": batch_size
    }
print(type(lr))
print(lr)
finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data['seq'], train_data['Y'], valid_data['seq'], valid_data['Y'], \
        seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = lr, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 512, final_lr = lr / 10, callbacks = training_callbacks)


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█▁▁▂▃▃▃▄▅▅▅▆▇▁
loss,█▄▃▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
lr,████████████▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▂▃▂▁▂▃▁▄▃▂▁▂▃▂▂▃▁▁▁▂▁▂▁▂▁▁
best_epoch,5
best_val_loss,0.45819
epoch,0
loss,0.35927
lr,1e-05
val_loss,0.48604


wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


<class 'float'>
0.0001
[2022_04_15-14:33:47] Training set: Filtered out 0 of 1291 (0.0%) records of lengths exceeding 510.
[2022_04_15-14:33:48] Validation set: Filtered out 0 of 130 (0.0%) records of lengths exceeding 510.
[2022_04_15-14:33:48] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
11/11 [==============================] - 10s 281ms/step - loss: 0.7999 - val_loss: 0.8898


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/100
11/11 [==============================] - 1s 108ms/step - loss: 0.5859 - val_loss: 0.6600
Epoch 3/100
11/11 [==============================] - 1s 109ms/step - loss: 0.5152 - val_loss: 0.5327
Epoch 4/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4500 - val_loss: 0.5070
Epoch 5/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4278 - val_loss: 0.4914
Epoch 6/100
11/11 [==============================] - 1s 110ms/step - loss: 0.4082 - val_loss: 0.5343
Epoch 7/100
11/11 [==============================] - 1s 107ms/step - loss: 0.4290 - val_loss: 0.5193
Epoch 8/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4161 - val_loss: 0.5199

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 9/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4100 - val_loss: 0.4779
Epoch 10/100
11/11 [==============================] - 1s 111ms/step - loss: 0.4006 - val_loss: 0.4866
Epoch 11/

11/11 [==============================] - 11s 418ms/step - loss: 0.4045 - val_loss: 0.4907
Epoch 2/100
11/11 [==============================] - 3s 247ms/step - loss: 0.3969 - val_loss: 0.4714
Epoch 3/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3894 - val_loss: 0.4817
Epoch 4/100
11/11 [==============================] - 3s 248ms/step - loss: 0.3789 - val_loss: 0.4876
Epoch 5/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3825 - val_loss: 0.4609
Epoch 6/100
11/11 [==============================] - 3s 248ms/step - loss: 0.3693 - val_loss: 0.4934
Epoch 7/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3705 - val_loss: 0.5067
Epoch 8/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3528 - val_loss: 0.4917

Epoch 00008: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Epoch 9/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3563 - val_loss: 0.4596
Epoch 10/100
11/11 [=

11/11 [==============================] - 11s 418ms/step - loss: 0.3270 - val_loss: 0.4616


In [90]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
        start_seq_len = 512, start_batch_size = 32)
print('Confusion matrix:')
display(confusion_matrix)

Confusion matrix:


,0,1
0,104,2
1,21,3


In [91]:
fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
f1

0.20689655172413793

In [83]:
len(train_data)

1291

In [86]:
len(test_data_old)

119

In [87]:
len(test_data)

156